<a href="https://colab.research.google.com/github/Amlan05/Amlan05/blob/main/Stragegy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
pip install pandas-ta

In [20]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import numpy as np

In [21]:
file_path = '/content/GODR Historical Data.csv'

In [22]:
data = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)

In [23]:
data.rename(columns={'Price': 'Close', 'Vol.': 'Volume', 'Change %': 'Change_Percentage'}, inplace=True)

In [24]:
def convert_volume(volume):
    if isinstance(volume, str):
        if 'K' in volume:
            return float(volume.replace('K', '').replace(',', '')) * 1_000
        elif 'M' in volume:
            return float(volume.replace('M', '').replace(',', '')) * 1_000_000
        else:
            return float(volume.replace(',', ''))
    return volume

# Apply the conversion
data['Volume'] = data['Volume'].apply(convert_volume)

In [25]:
data = data.sort_values(by='Date').reset_index(drop=True)

In [26]:
print(data.tail())

           Date    Close     Open     High      Low     Volume  \
1002 2025-01-16  2363.65  2374.00  2418.40  2335.95   677420.0   
1003 2025-01-17  2407.60  2362.00  2421.65  2336.00   505100.0   
1004 2025-01-20  2428.15  2427.45  2445.00  2392.05   521310.0   
1005 2025-01-21  2365.85  2440.00  2446.45  2359.00   806390.0   
1006 2025-01-22  2239.25  2365.85  2378.40  2200.15  1090000.0   

     Change_Percentage  
1002             2.46%  
1003             1.86%  
1004             0.85%  
1005            -2.57%  
1006            -5.35%  


In [27]:
# Calculate Support and Resistance
data['Support'] = data['Low'][data['Low'] == data['Low'].rolling(window=9, center=True).min()]
data['Resistance'] = data['High'][data['High'] == data['High'].rolling(window=9, center=True).max()]

# Fill missing values with the previous value (forward-fill)
data['Support'] = data['Support'].ffill()
data['Resistance'] = data['Resistance'].ffill()

In [28]:
# RSI with SMA
data['RSI'] = ta.rsi(data['Close'], length=14, mamode="sma")

In [29]:
# # MACD
macd_data = ta.macd(data['Close'], fast=12, slow=26, signal=9, append=True)
data['MACD_Histogram'] = macd_data['MACDh_12_26_9']  # Histogram (MACD - Signal line)

In [30]:
# ATR
data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)

In [31]:
# Bollinger Bands
bollinger = ta.bbands(data['Close'], length=20, std=2)
data['Upper_BB'] = bollinger['BBU_20_2.0']
data['Lower_BB'] = bollinger['BBL_20_2.0']
data['Middle_BB'] = bollinger['BBM_20_2.0']

In [32]:
# Moving Averages
data['100_MA'] = ta.sma(data['Close'], length=100)
data['20_MA'] = ta.sma(data['Close'], length=20)

In [33]:
# Rolling Volume Mean
data['Rolling_Volume_Mean'] = data['Volume'].rolling(window=20).mean()

In [34]:
data.tail()

,Date,Close,Open,High,Low,Volume,Change_Percentage,Support,Resistance,RSI,MACD_Histogram,ATR,Upper_BB,Lower_BB,Middle_BB,100_MA,20_MA,Rolling_Volume_Mean
1002,2025-01-16,2363.65,2374.00,2418.40,2335.95,677420.0,2.46%,2240.05,3015.0,30.696350,-43.605243,97.237027,3106.784354,2208.730646,2657.7575,2867.8390,2657.7575,552767.0
1003,2025-01-17,2407.60,2362.00,2421.65,2336.00,505100.0,1.86%,2240.05,3015.0,34.717629,-31.507018,96.409382,3064.279192,2193.590808,2628.9350,2863.0380,2628.9350,542935.0
1004,2025-01-20,2428.15,2427.45,2445.00,2392.05,521310.0,0.85%,2240.05,3015.0,36.570880,-19.972784,93.305141,3038.184290,2176.905710,2607.5450,2858.3765,2607.5450,536452.0
1005,2025-01-21,2365.85,2440.00,2446.45,2359.00,806390.0,-2.57%,2240.05,3015.0,33.468888,-14.638464,92.886916,3004.255371,2158.679629,2581.4675,2853.1580,2581.4675,556704.5
1006,2025-01-22,2239.25,2365.85,2378.40,2200.15,1090000.0,-5.35%,2240.05,3015.0,28.228908,-17.539875,98.984279,2977.122008,2123.297992,2550.2100,2846.8595,2550.2100,598095.5


In [35]:
# Price Action Logic
def detect_price_action(row):
    body = abs(row['Close'] - row['Open'])
    wick_up = row['High'] - max(row['Close'], row['Open'])
    wick_down = min(row['Close'], row['Open']) - row['Low']

    if wick_up > 2 * body and wick_down < body and row['Close'] < row['Open']:
        return 'Shooting Star'
    elif wick_down > 2 * body and wick_up < body and row['Close'] > row['Open']:
        return 'Hammer'
    elif body > wick_up + wick_down and row['Volume'] > row['Rolling_Volume_Mean']:
        if row['Close'] < row['Open']:
            return 'Large Body Down'
        elif row['Close'] > row['Open']:
            return 'Large Body Up'
    return None

data['Price_Action'] = data.apply(detect_price_action, axis=1)

In [36]:
# Backtesting Logic
position = 0
entry_price = None
stop_loss_price = None
above_ma_volume_counter = 0
exit_candle = 0
data['Buy_Date'] = None
data['Sell_Date'] = None
data['Stop_Loss'] = None

In [38]:
# Function to find an entry point
def find_entry(data, start_index, position=0):
    for index in range(start_index, len(data)):
        row = data.iloc[index]

        # Entry condition for long
        if (
            position == 0
            and row['RSI'] >=60
            and row['MACD_Histogram'] > 0
            and row['Open'] < row['100_MA']
            and row['Close'] > row['20_MA']
            and row['Close'] > row['Support']
        ):
            entry_price = row['Close']
            entry_date = row['Date']
            stop_loss_price = entry_price - 1.5 * row['ATR']
            return index, entry_date, entry_price, stop_loss_price, 1

        # Entry condition for short
        if(
            position == 0
            and row['RSI'] <= 40
            and row['MACD_Histogram'] < 0
            and row['Open'] > row['100_MA']
            and row['Close'] < row['20_MA']
            and row['Close'] < row['Resistance']
        ):
            entry_price = row['Close']
            entry_date = row['Date']
            stop_loss_price = entry_price + 1.5 * row['ATR']
            return index, entry_date, entry_price, stop_loss_price, -1

    return None, None, None, None, 0

In [39]:
def find_exit(data, entry_index, entry_price, stop_loss_price, position):
    support = float('nan')
    resistance = float('nan')
    above_ma_volume_counter = 0
    exit_candle = 0

    for index in range(entry_index + 1, len(data)):
        row = data.iloc[index]
        prevRow = data.iloc[index - 1] if index > 0 else None

        # Update support and resistance levels
        if prevRow is not None:
            if pd.notna(prevRow['Support']):
                support = prevRow['Support']
            if pd.notna(prevRow['Resistance']):
                resistance = prevRow['Resistance']

        # Exit logic for Buy position
        if position == 1:
            # Stop loss hit
            if row['Close'] <= stop_loss_price:
                return index, row['Date'], row['Close']

            # Case 1: Rejection at resistance or 100 MA
            elif (row['High'] >= resistance and row['Close'] < resistance) or \
                 (row['High'] > row['100_MA'] and row['Close'] < row['100_MA']):
                return index, row['Date'], row['Close']

            # Case 2: High volume red candles
            elif (row['Open'] > row['100_MA'] or row['Open'] > resistance) and \
                 (row['Close'] < row['100_MA'] or row['Close'] < resistance):
                if row['Volume'] > row['Rolling_Volume_Mean']:
                    above_ma_volume_counter += 1
                elif row['Volume'] <= row['Rolling_Volume_Mean']:
                    exit_candle += 1

            # Exit after consecutive high volume candles
            elif above_ma_volume_counter >= 1 and (row['Open'] < row['100_MA'] or row['Open'] < resistance):
                return index, row['Date'], row['Open']

            # Case 3: Low volume green candles
            elif (row['Open'] < row['100_MA'] or row['Open'] < resistance) and exit_candle >= 1:
                return index, row['Date'], row['Close']

        # Exit logic for Sell position
        if position == -1:
            # Stop loss hit
            if row['Close'] >= stop_loss_price:
                return index, row['Date'], row['Close']

            # Case 1: Rejection at support or 100 MA
            elif (row['Low'] <= support and row['Close'] > support) or \
                 (row['Low'] < row['100_MA'] and row['Close'] > row['100_MA']):
                return index, row['Date'], row['Close']

            # Case 2: High volume green candles
            elif (row['Open'] < row['100_MA'] or row['Open'] < support) and \
                 (row['Close'] > row['100_MA'] or row['Close'] > support):
                if row['Volume'] > row['Rolling_Volume_Mean']:
                    above_ma_volume_counter += 1
                elif row['Volume'] <= row['Rolling_Volume_Mean']:
                    exit_candle += 1

            # Exit after consecutive high volume candles
            elif above_ma_volume_counter >= 1 and (row['Open'] > row['100_MA'] or row['Open'] > support):
                return index, row['Date'], row['Open']

            # Case 3: Low volume green candles
            elif (row['Open'] > row['100_MA'] or row['Open'] > support) and exit_candle >= 1:
                return index, row['Date'], row['Close']

    # If no exit is found within the dataset
    return None, None, None

In [41]:
# Main loop to calculate PnL and track trades
def calculate_pnl(data):
    position = 0  # 0 means no position, 1 means Buy
    start_index = 10
    total_pnl = 0
    num_trades = 0
    profit_trades = 0

    while start_index < len(data):
        # Find entry
        entry_index, entry_date, entry_price, stop_loss_price, position = find_entry(data, start_index, position)
        if entry_index is None:  # No more entries
            break

        print(f"Entry: {entry_date}, Price: {entry_price}")

        # Find exit
        exit_index, exit_date, exit_price = find_exit(data, entry_index, entry_price, stop_loss_price, position)
        if exit_index is None:  # No exit found
            break

        # Calculate PnL for the trade
        pnl = entry_price - exit_price
        if(pnl > 0):
            profit_trades+=1
        total_pnl += pnl
        num_trades += 1

        print(f"Exit: {exit_date}, Price: {exit_price}, PnL: {pnl}")

        # Update start_index to resume search after exit
        start_index = exit_index + 1
        position = 0  # Reset position after exit

    print(f"Total PnL: {total_pnl}, Number of Trades: {num_trades}, Succesful Trades: {profit_trades}")
    return total_pnl, num_trades

# Run the calculation
calculate_pnl(data)

Entry: 2021-06-02 00:00:00, Price: 1390.65
Exit: 2021-06-21 00:00:00, Price: 1360.0, PnL: 30.65000000000009
Entry: 2021-08-20 00:00:00, Price: 1484.95
Exit: 2021-08-26 00:00:00, Price: 1440.0, PnL: 44.950000000000045
Entry: 2021-11-22 00:00:00, Price: 2116.2
Exit: 2021-12-02 00:00:00, Price: 2020.75, PnL: 95.44999999999982
Entry: 2021-12-03 00:00:00, Price: 2020.1
Exit: 2022-01-21 00:00:00, Price: 1798.85, PnL: 221.25
Entry: 2022-03-30 00:00:00, Price: 1665.6
Exit: 2022-04-05 00:00:00, Price: 1672.05, PnL: -6.4500000000000455
Entry: 2022-04-06 00:00:00, Price: 1681.65
Exit: 2022-04-19 00:00:00, Price: 1565.95, PnL: 115.70000000000005
Entry: 2022-07-15 00:00:00, Price: 1343.55
Exit: 2022-07-20 00:00:00, Price: 1412.8, PnL: -69.25
Entry: 2022-07-21 00:00:00, Price: 1441.6
Exit: 2022-07-26 00:00:00, Price: 1406.75, PnL: 34.84999999999991
Entry: 2022-07-27 00:00:00, Price: 1441.3
Exit: 2022-08-04 00:00:00, Price: 1391.4, PnL: 49.899999999999864
Entry: 2023-04-10 00:00:00, Price: 1228.35
Ex

(510.4499999999996, 17)